## Importing libraries

In [1]:
import pandas as pd
import numpy as np 
from autoviml.Auto_ViML import Auto_ViML

Imported Auto_ViML version = 0.1.463. 
             m, feats, trainm, testm = Auto_ViML(train, target, test, 
                                    sample_submission='',
                                    scoring_parameter='',
                                    hyper_param='GS',feature_reduction=True,
                                     Boosting_Flag=None,Binning_Flag=False,
                                    Add_Poly=0, Stacking_Flag=False,Imbalanced_Flag=False, 
                                    verbose=0)
            
To remove previous versions, perform 'pip uninstall autoviml'


In [13]:
url1 = '../dataset/dataset_treino.csv'
url2 = '../dataset/dataset_teste.csv'
url3 = '../dataset/sample_submission.csv'
train = pd.read_csv(url1,index_col=None)
test = pd.read_csv(url2,index_col=None)
sample_submission = pd.read_csv(url3,index_col=None)
print(train.shape,test.shape,sample_submission.shape)

(114321, 133) (114393, 132) (114393, 2)


In [14]:
sample_submission = ''
scoring_parameter = 'logloss'
target='target'

In [15]:
train.fillna(train.mean(),inplace=True)
test.fillna(test.mean(),inplace=True)

In [16]:
train['v1'].describe()

count    1.143210e+05
mean     1.630686e+00
std      8.132649e-01
min     -9.996497e-07
25%      1.346153e+00
50%      1.630686e+00
75%      1.630686e+00
max      2.000000e+01
Name: v1, dtype: float64

In [21]:
from scipy.stats import mstats

# Tratando os dados outliers com Winsorize
for col in train.columns:
    if train[col].dtype !='object':
        train[col] = mstats.winsorize(train[col], limits=[0.05, 0.05])[0]

In [29]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [30]:
train['v100'].describe()

count   114321.00000
mean        19.47020
std          0.00000
min         19.47020
25%         19.47020
50%         19.47020
75%         19.47020
max         19.47020
Name: v100, dtype: float64

In [ ]:
numerical_feats = train.dtypes[train.dtypes != "object"].index
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = train.dtypes[train.dtypes == "object"].index
print("Number of Categorical features: ", len(categorical_feats))

In [ ]:
# Função para criação de novas features, agrupando por v2
def ft(data):
    df = pd.DataFrame()
    
    for col in numerical_feats:
        if col in ['ID','target','v2']:
            continue
        
        #print(col)
        
        df[col + '_mean']   = data.groupby(['v2'])[col].mean()
        df[col + '_median'] = data.groupby(['v2'])[col].median()
        df[col + '_max']    = data.groupby(['v2'])[col].max()
        df[col + '_min']    = data.groupby(['v2'])[col].min()

    return df

In [ ]:
new_train_ft = ft(train)
new_test_ft = ft(test)

In [ ]:
new_train_ft = new_train_ft.reset_index()
new_test_ft = new_test_ft.reset_index()
new_train_ft.shape, new_test_ft.shape

In [ ]:
new_train_ft.head()

In [ ]:
train_merge = pd.merge(train, new_train_ft, on='v2',how='left')
train_merge.head()

In [ ]:
test_merge = pd.merge(test, new_test_ft, on='v2',how='left')
test_merge.head()

In [ ]:
train_merge.shape, test_merge.shape

In [ ]:
#### If Boosting_Flag = True => XGBoost, False=>ExtraTrees, None=>Linear Model
m, feats, trainm, testm = Auto_ViML(train_merge, target, test_merge, sample_submission,
                                    scoring_parameter=scoring_parameter,
                                    hyper_param='GS',
                                    feature_reduction=True,
                                    Boosting_Flag=None,
                                    Binning_Flag=True,
                                    Add_Poly=0, 
                                    Stacking_Flag=False,                                    
                                    Imbalanced_Flag=True, 
                                    verbose=1)

In [ ]:
#### If Boosting_Flag = True => XGBoost, False=>ExtraTrees, None=>Linear Model
m, feats, trainm, testm = Auto_ViML(train_merge, target, test_merge, sample_submission,
                                    scoring_parameter=scoring_parameter,
                                    hyper_param='GS',
                                    feature_reduction=True,
                                    Boosting_Flag=False,
                                    Binning_Flag=True,
                                    Add_Poly=0, 
                                    Stacking_Flag=False,                                    
                                    Imbalanced_Flag=True, 
                                    verbose=1)